**References**

[1] Zhang, M., Wu, B., Yu, M., Zou, W., & Zheng, Y. (2014). Crop Condition Assessment with Adjusted NDVI Using the Uncropped Arable Land Ratio. Remote Sensing, 6(6), 5774-5794. https://doi.org/10.3390/rs6065774

In [7]:
# Global Variables
# Region for processing
district_name = 'Aligarh'
country_name = 'India'
#NDVI Threshold (See references)
ndvi_threshold = 0.4
# Date range (Rabi cropping season)
start_date = '2022-10-01'
end_date   = '2023-03-01'

In [10]:
# ===========================================================
# Agriculture Extent Mask for Aligarh District (India)
# Using FAO GAUL Boundaries + Sentinel-2 NDVI Thresholding
# Google Earth Engine Python API (Colab)
# ===========================================================

# -------------------------------
# 1. Authenticate and Initialize Earth Engine
# -------------------------------
import ee
import geemap
from google.colab import userdata
EE_PROJECT_NAME = userdata.get('EE_PROJECT_NAME')

# Trigger authentication flow.
ee.Authenticate()

# Initialize the Earth Engine library.
ee.Initialize(project=EE_PROJECT_NAME)

# -------------------------------
# 2. Import FAO GAUL Admin Boundaries
# -------------------------------
admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level2")

# Select Aligarh district (India)
aligarh_district = admin_boundaries \
    .filter(ee.Filter.eq('ADM2_NAME', district_name)) \
    .filter(ee.Filter.eq('ADM0_NAME', country_name))

district_geometry = aligarh_district.geometry()

# ROI Visualization
Map = geemap.Map(center=[27.9, 78.0], zoom=10)
Map.add_basemap('HYBRID')
Map.addLayer(district_geometry, {}, district_name)
Map.addLayerControl()
Map


Map(center=[27.9, 78.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [12]:
# ------------------------------------
# 3. ESA Sentinel-2 Data Preprocessing
# ------------------------------------

# Load Sentinel-2 SR harmonized dataset
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(district_geometry)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .select(['B4', 'B8']))  # Red = B4, NIR = B8

# Create median composite
s2_composite = s2_collection.median().clip(district_geometry)

# -------------------------------
# 4. Compute NDVI
# -------------------------------
ndvi = s2_composite.normalizedDifference(['B8','B4']).rename('NDVI')

# -------------------------------------------
# 5. Crop Extent Mask (NDVI > ndvi_threshold)
# -------------------------------------------
crop_extent = ndvi.gt(ndvi_threshold).selfMask()  # Keep only growing crops
ndvi_vis = {'min': 0, 'max': 1, 'palette': ['brown','yellow','green']}

# NDVI visualization

Map.addLayer(ndvi, ndvi_vis, "NDVI (Median)")
Map.addLayer(crop_extent, {'palette':['green']}, "Crop Extent (using NDVI)")
Map.addLayerControl()
#Map

In [18]:

# -------------------------------
# 6. Area Calculation
# -------------------------------
# Total district area in sq.km
total_district_area = district_geometry.area().divide(1e6).round()
print('District Area (sq.km):', total_district_area.getInfo())
print('District Area (in Ha):', total_district_area.multiply(100).getInfo())

# Crop area in hectares
crop_area = crop_extent.multiply(ee.Image.pixelArea())
crop_area_ha = crop_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=district_geometry,
    scale=10,
    maxPixels=1e13
).get('NDVI')

print('Estimated Cropland Area (Ha):', ee.Number(crop_area_ha).divide(10000).getInfo())
print('Percentage of Cropland Area in Distric:', ee.Number(crop_area_ha).divide(10000).divide(total_district_area).divide(100).multiply(100).getInfo())

District Area (sq.km): 3871
District Area (in Ha): 387100
Estimated Cropland Area (Ha): 304866.7694389687
Percentage of Cropland Area in Distric: 78.75659246679635
